In [2]:
import bokeh

In [3]:

import math
def InitialDataReducedMass(orbitalangle,r0):
    phi=orbitalangle
    radiusseparation=r0 #separation one hundred
    eccentricity=np.zeros(2)
    mass=np.ones(2)
    masssun=1.989*10**30
    lsun=3.828*10**26
    massearth=5.9722*10**24
    masses=mass #*masssun (natural units)
    return phi, radiusseparation,eccentricity, masses
    

In [4]:
def getxyuveqreducedmass(initdat): #using reduced mass cooordinate system
    phi,radiusseparation,eccentricity, masses=initdatreducedmass
    #print(orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=1
    Gconstant=1
    x0=np.zeros(2)
    y0=np.zeros(2)
    x0[1]=radiusseparation*np.cos(phi)*metersperAU
    y0[1]=radiusseparation*np.sin(phi)*metersperAU
    if phi==0:
        print("zero")
        x0[1]=radiusseparation
        y0[1]=0
    if phi==math.pi:
        print("pi")
        x0[1]=-radiusseparation
        y0[1]=0
    z0=np.zeros(2)
    print("x0", x0)
    print("y0", y0)
    
    #start at perihelion for both (eliptical, doesn't generalize to three body)
    #actually start with circular orbit
    ux0=np.zeros(2) #*149597870700
    #centrepital force balances gravitational force
    reducedmass=np.zeros(2)
    masstotal=np.sum(masses)
    reducedmass[0]=masstotal
    print(masses)
    for i in np.arange(1):
        j=(i+1)%2 #reverse masses
        reducedmass[i+1]=masses[i]*masses[j]/np.sum(masses)
    print(reducedmass)
    metersperAU=1 #natural units
    rphys=np.zeros(len(masses))
    for i in np.arange(2):
        #rphys is r in the reduced mass system
        rphys[i]=radiusseparation #*metersperAU*reducedmass[i]/masses[i]
    x0[0]=0.0
    x0phys=rphys*x0/radiusseparation
    x0phys[0]=0.0
    y0phys=rphys*y0/radiusseparation
    z0phys=np.zeros(2)
    r0=np.sqrt(x0**2+y0**2)
    print(rphys)
    #G=1
    Gconstant=1
    F=(Gconstant*masstotal**2/(2*rphys)**2)
    print(F)
    #centF=reducedmass*v**2/rphys
    #centF=accel
    v=np.sqrt(Gconstant*masstotal/(rphys))
    print(v)
    print("r0", r0)
    ux0=-v*y0/radiusseparation #[1]
    uy0=v*x0/radiusseparation #[1]
        
    #ux0=-v*y0/r0
    #uy0=v*x0/r0 #initial data in y only
    #evolve in plane only
    #circular orbit
    uz0=np.zeros(2)
    
    a=Gconstant*masstotal/((2*rphys)**2)
    ax0=-a*x0/radiusseparation
    ay0=-a*y0/radiusseparation
    az0=np.zeros(2)
    
    statevec=[]
    avec=[]
    for i in np.arange(len(x0phys)):
        stateveci=np.array([x0phys[i],y0phys[i],z0phys[i],ux0[i],uy0[i],uz0[i]])
        aveci=np.array([ax0[i],ay0[i],az0[i]])
        statevec.append(stateveci)
        avec.append(aveci)
    statevecnp=np.array(statevec)
    avecnp=np.array(avec)
    return masstotal, reducedmass,statevecnp,avecnp

In [5]:
import random,numpy as np
initdatreducedmass=InitialDataReducedMass(0,50)

In [6]:
xyuvaeqreducedmass=getxyuveqreducedmass(initdatreducedmass)
print(xyuvaeqreducedmass)
masstotal, reducedmass,xvec,avec=xyuvaeqreducedmass

zero
x0 [ 0. 50.]
y0 [0. 0.]
[1. 1.]
[2.  0.5]
[50. 50.]
[0.0004 0.0004]
[0.2 0.2]
r0 [ 0. 50.]
(2.0, array([2. , 0.5]), array([[ 0. ,  0. ,  0. , -0. ,  0. ,  0. ],
       [50. ,  0. ,  0. , -0. ,  0.2,  0. ]]), array([[-0.    , -0.    ,  0.    ],
       [-0.0002, -0.    ,  0.    ]]))


In [7]:
xvec[:,2]

array([0., 0.])

In [8]:
axii=np.zeros(len(xvec[:,0]))
rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
rii

array([ 0., 50.])

In [9]:
import RK4implicit

In [10]:
class OrbitDiffEq:
    def __init__(self,reducedmass,xvec,avec,t0):
        self.reducedmass=reducedmass
        self.xvec=xvec
        self.ti=t0
        self.avec=avec
        self.reducedmass0=reducedmass
        self.xvec0=xvec
        self.t0=t0
        self.avec0=avec
    def reset(self):
        self.reducedmass=self.reducedmass0
        self.xvec=self.xvec0
        self.ti=self.t0
        self.avec=self.avec0
    def dxidt(self,t,xvec):
        return xvec[:,3].transpose()
    def dyidt(self,t,xvec):
        return xvec[:,4].transpose()
    def dzidt(self,t,xvec):
        return xvec[:,5].transpose()
    def dvxidt(self,t,xvec):
        #return axi[m]
        axii=np.zeros(len(xvec[:,0]))
        rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
        Gconstant=1 #6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(1,len(rii)):
                if j==0:
                    axii[j]=0.0
                if j!=k:
                    rreljk=np.abs((xvec[j,0] - xvec[k,0])**2+(xvec[j,1]-xvec[k,1])**2+(xvec[j,2]-xvec[k,2])**2)**(1./2.)
                    axii[j]-=Gconstant*self.reducedmass[k]*(xvec[j,0]  - xvec[k,0])/rreljk**3
        return axii
    def dvyidt(self,t,xvec):
        #return axi[m]
        ayii=np.zeros(len(xvec[:,0]))
        rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
        Gconstant=1 #6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(1,len(rii)):
                if j==0:
                    ayii[j]=0.0
                if j!=k:
                    rreljk=np.abs((xvec[j,0] - xvec[k,0])**2+(xvec[j,1]-xvec[k,1])**2+(xvec[j,2]-xvec[k,2])**2)**(1./2.)
                    ayii[j]-=Gconstant*self.reducedmass[k]*(xvec[j,1]  - xvec[k,1])/rreljk**3
        return ayii
    def dvzidt(self,t,xvec):
        #return axi[m]
        azii=np.zeros(len(xvec[:,0]))
        rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
        Gconstant=1 #6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(1,len(rii)):
                if j==0:
                    azii[j]=0.0
                if j!=k:
                    rreljk=np.abs((xvec[j,0] - xvec[k,0])**2+(xvec[j,1]-xvec[k,1])**2+(xvec[j,2]-xvec[k,2])**2)**(1./2.)
                    azii[j]-=Gconstant*self.reducedmass[k]*(xvec[j,2]  - xvec[k,2])/rreljk**3
        return azii
    
    def dvecdt(self,t,xvec):
        avec=np.array([self.dvxidt(self,xvec),self.dvyidt(self,xvec),self.dvzidt(self,xvec)])
        avecT=avec.transpose()
        self.avec=avecT
        dvec2=np.array([self.dxidt(t,xvec),self.dyidt(t,xvec),self.dzidt(self,xvec),avec[0,:],avec[1,:],avec[2,:]])
                       
        dvec2T=dvec2.transpose()
        return dvec2T
    def updateINTERNAL(self,xvecii,tii):
        self.xvec=xvecii
        self.ti=tii
        return self
    def update(self,xvecii,avecii,tii):
        self.xvec=xvecii
        self.avec=avecii,
        self.ti=tii
    def print2D(self):
        print(self.reducedmass,self.xvec[1], self.ti)
        return self
    def list2D(self):
        return self.reducedmass,self.xvec,self.avec,self.ti
    def timestepRK4ODE(self,step,dt):

    
        h=dt
        #tnew,ynew, intval=RK4(h,t,y,f)
        #m represents choices of mass
        i=step
        
        tnew,intvalxvec=RK4implicit.RK4implicit(h,self.ti,self.xvec,self.dvecdt)
        #xii = intvalx
        #tnew,intvalvx=RK4implicit(h,self.ti,self.vxi,self.dvxidt)
        #vxii=intvalvx
        #tnew,intvaly=RK4implicit(h,self.ti,self.yi,self.dyidt)
        #yii = intvaly
        #tnew,intvalvy=RK4implicit(h,self.ti,self.vyi,self.dvyidt)
        #vyii=intvalvy
        #tnew,intvalz=RK4implicit(h,self.ti,self.zi,self.dzidt)
        #zii = intvalz
        #tnew,intvalvz=RK4implicit(h,self.ti,self.vzi,self.dvzidt)
        #vzii=intvalvz
        #axii=self.dvxidt(self.ti,np.zeros(np.shape(self.xi)))
        #ayii=self.dvyidt(self.ti,np.zeros(np.shape(self.yi)))
        #azii=self.dvzidt(self.ti,np.zeros(np.shape(self.zi)))
 
        #print(xii)
        self.updateINTERNAL(intvalxvec,tnew)
        return reducedmass, self.xvec,self.avec,self.ti

In [11]:
mtotal,reducedmass,xvec,avec=xyuvaeqreducedmass
ODEeq= OrbitDiffEq(reducedmass,xvec,avec,0.0)
ODEeq.print2D()
ODEeq.print2D()

[2.  0.5] [50.   0.   0.  -0.   0.2  0. ] 0.0
[2.  0.5] [50.   0.   0.  -0.   0.2  0. ] 0.0


In [33]:
dt=.1 #natural units #*31556926 #seconds per year
numsteps=20000
mtotal, reducedmass,xvec0,avec0=xyuvaeqreducedmass
ODEeq= OrbitDiffEq(reducedmass,xvec0,avec0,0.0)
ODEeq.print2D()
t=0.0
star1x=[]
star2x=[]
star1a=[]
star2a=[]
times=[]
for i in np.arange(1,numsteps):
    reducedmass,xvec,avec,t=ODEeq.timestepRK4ODE(i,dt)
    if i< 10:
        ODEeq.print2D()
    star1x.append(xvec[0])
    star2x.append(xvec[1])
    star1a.append(avec[0])
    star2a.append(avec[1])
    times.append(t)



[2.  0.5] [50.   0.   0.  -0.   0.2  0. ] 0.0
[2.  0.5] [ 4.99999996e+01  2.00000000e-02  0.00000000e+00 -8.00000000e-05
  1.99999998e-01  0.00000000e+00] 0.1
[2.  0.5] [ 4.99999912e+01  3.99999997e-02  0.00000000e+00 -1.59999981e-04
  1.99999965e-01  0.00000000e+00] 0.2
[2.  0.5] [ 4.99999748e+01  5.99999958e-02  0.00000000e+00 -2.39999931e-04
  1.99999899e-01  0.00000000e+00] 0.30000000000000004
[2.  0.5] [ 4.99999504e+01  7.99999853e-02  0.00000000e+00 -3.19999837e-04
  1.99999802e-01  0.00000000e+00] 0.4
[2.  0.5] [ 4.99999180e+01  9.99999648e-02  0.00000000e+00 -3.99999686e-04
  1.99999672e-01  0.00000000e+00] 0.5
[2.  0.5] [ 4.99998776e+01  1.19999931e-01  0.00000000e+00 -4.79999465e-04
  1.99999510e-01  0.00000000e+00] 0.6
[2.  0.5] [ 4.99998292e+01  1.39999881e-01  0.00000000e+00 -5.59999162e-04
  1.99999317e-01  0.00000000e+00] 0.7
[2.  0.5] [ 4.99997728e+01  1.59999812e-01  0.00000000e+00 -6.39998763e-04
  1.99999091e-01  0.00000000e+00] 0.7999999999999999
[2.  0.5] [ 4.99997

In [34]:
npstar2x=np.array(star2x)
npstar1x=np.array(star1x)
npstar2a=np.array(star2a)
npstar1a=np.array(star1a)
nptimes=np.array(times)

In [35]:
npstar2x

array([[ 4.99999996e+01,  2.00000000e-02,  0.00000000e+00,
        -8.00000000e-05,  1.99999998e-01,  0.00000000e+00],
       [ 4.99999912e+01,  3.99999997e-02,  0.00000000e+00,
        -1.59999981e-04,  1.99999965e-01,  0.00000000e+00],
       [ 4.99999748e+01,  5.99999958e-02,  0.00000000e+00,
        -2.39999931e-04,  1.99999899e-01,  0.00000000e+00],
       ...,
       [-5.60444834e+00,  4.99458624e+01,  0.00000000e+00,
        -1.98305872e-01, -2.21322612e-02,  0.00000000e+00],
       [-5.62427889e+00,  4.99436488e+01,  0.00000000e+00,
        -1.98297041e-01, -2.22109438e-02,  0.00000000e+00],
       [-5.64410855e+00,  4.99414273e+01,  0.00000000e+00,
        -1.98288179e-01, -2.22896229e-02,  0.00000000e+00]])

In [36]:
max(npstar2x[:,1])

50.25514773127106

In [37]:
min(npstar2x[:,1])

-50.19673329518243

Orbit doesn't go out to 53

In [38]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="Circular orbit, RK4", plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(npstar1x[:,0],npstar1x[:,1], size=15, line_color="purple",fill_color="purple", fill_alpha=0.5), #legend="M")
p.line(npstar2x[:,0],npstar2x[:,1], line_color="blue"), #legend="mu")


#p.legend.location = "bottom_left"
#p.legend.click_policy="hide"

show(p) # show the

Loading BokehJS ...

%%html
<a href="https://github.com/sdorsher/ManyBodyProblem/blob/master/reduced%20mass%20circular%20orbit%20rk4.png"> Circular orbit in the reduced mass system </a> 
Note that the central heavier mass is stationary in this reference frame with the smaller lighter mass orbits about it in a circular orbit. 

In [39]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="X-acceleration, Circular orbit, RK4", plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(times,npstar1a[:,0], size=1, line_color="purple",fill_color="purple", fill_alpha=0.5) #legend="M")
p.circle(times,npstar2a[:,0], size=1, line_color="blue") #legend="mu")

#p.legend.location = "bottom_left"
#p.legend.click_policy="hide"

show(p) # show the

Loading BokehJS ...

%%html
<a href="https://github.com/sdorsher/ManyBodyProblem/blob/master/reduced%20mass%20acceleration%20circular%20RK4.png">Acceleration in the reduced mass reference frame for a circular orbit </a> 

In the reduced mass reference frame, the acceleration of the larger mass is zero and the acceleration of the x componenet of the smaller mass is sinusiodal. 

In [30]:
def RunFirstOrbit(dt,initcondition,printoutput):
    masstotal,mass0,xvec, avec=initcondition
    ODEeq.reset()
    x0=xvec[1,0]
    y0=xvec[1,1]
    #lastvalue=np.array([1.,1.])
    #thisvalue=np.array([xvec[1,0],xvec[1,1]])
    lastvalue=[1.,1.]
    thisvalue=[xvec[1,0],xvec[1,1]]
    i=1
    t=0
    while not (lastvalue[0]>0. and lastvalue[1]<0. and thisvalue[0]> 0. and thisvalue[1]>=0.):
        reducedmass,xvec,avec,t=ODEeq.timestepRK4ODE(i,dt)
        #star2x=xvec[1]
        #if printoutput:
        #    print(xvec[1,0])
        #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
        #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
        #print(ax)
        #xy=np.array([star2x[0],star2x[1]])
        #thisvalue=lastvalue
        #lastvalue=xy
        xy=[xvec[1,0],xvec[1,1]]
        lastvalue,thisvalue=thisvalue, xy
        #lastvalue=thisvalue
        #thisvalue[0]=x[1]
        #thisvalue[1]=y[1]
        #thisvalue[0]=x[1]
        #thisvalue[1]=y[1]
        if printoutput:
            print(xvec[1,0],xvec[1,1],xy,lastvalue,thisvalue)
        i+=1
    #slope=(thisvalue[1]-lastvalue[1])/(thisvalue[0]-lastvalue[0])
    #xcrossing=1./slope*(0.-lastvalue[1])+lastvalue[0] #(y1-y2)=m(x1-x2)
    xcrossing=(lastvalue[0]+thisvalue[0])/2.
    print(dt, thisvalue[0], thisvalue[1], lastvalue[0], lastvalue[1], x0, y0,xcrossing,np.abs(x0-xcrossing))
    #interpolate where it crosses zero
    #slope=(thisvalue[1]-lastvalue[1])/(thisvalue[0]-lastvalue[0])
    #xcrossing=1./slope*(0.-lastvalue[1])+lastvalue[0] #(y1-y2)=m(x1-x2)
    return np.abs(x0-xcrossing)

In [41]:
dt=.1 #*31556926 #seconds per year
numsteps=2000
mtotal,mass0,xvec,avec=xyuvaeqreducedmass
L0arr=[]
dtarr=[]
dtinit=0.125/4.
#dtinit=.1
for j in np.arange(6):
    dt=(2**j)*dtinit
    dtarr.append(dt)
    L0=RunFirstOrbit(dt,xyuvaeqreducedmass,False)
    L0arr.append(L0)
print(dtarr, L0arr)

0.03125 50.07605655292996 0.0008054239264663763 50.076055890547345 -0.005439827882285399 50.0 0.0 50.076056221738654 0.07605622173865356
0.0625 50.147154028915054 0.00876248650905741 50.14715322928154 -0.0037191596485442596 50.0 0.0 50.1471536290983 0.1471536290982982
0.125 50.274512446009474 0.01855203345580556 50.27451025617781 -0.006379619231602199 50.0 0.0 50.27451135109364 0.27451135109364344
0.25 50.470134056754205 0.014913711978088037 50.4701069986311 -0.03485284822592706 50.0 0.0 50.47012052769266 0.4701205276926572
0.5 50.626334937493816 0.07063378572261625 50.62633143629279 -0.028745660831163228 50.0 0.0 50.626333186893305 0.6263331868933051
1.0 49.99998340688077 0.04073463839830538 49.99974634566594 -0.15926508777278675 50.0 0.0 49.99986487627336 0.00013512372664337136
[0.03125, 0.0625, 0.125, 0.25, 0.5, 1.0] [0.07605622173865356, 0.1471536290982982, 0.27451135109364344, 0.4701205276926572, 0.6263331868933051, 0.00013512372664337136]


In [42]:
dtarr

[0.03125, 0.0625, 0.125, 0.25, 0.5, 1.0]

In [43]:
L0arr

[0.07605622173865356,
 0.1471536290982982,
 0.27451135109364344,
 0.4701205276926572,
 0.6263331868933051,
 0.00013512372664337136]

In [55]:
xdat=np.log2(np.array(dtarr[0:5]))
ydat=np.log2(np.array(L0arr[0:5]))

In [56]:
xdat

array([-5., -4., -3., -2., -1.])

In [57]:
ydat

array([-3.71678992, -2.76460497, -1.86506229, -1.08889742, -0.67499777])

In [58]:
fitparam=np.polyfit(xdat,ydat,1)

In [59]:
fitparam

array([0.77592918, 0.30571708])

In [60]:
errfn=np.poly1d(fitparam)

In [61]:
errfn

poly1d([0.77592918, 0.30571708])

In [63]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="L0 error after one orbit, Circular, Vector, RK4", x_axis_label="dt", y_axis_label="L0", plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha

p.circle(np.log2(np.array(dtarr)),np.log2(np.array(L0arr)), line_color="blue") #legend="L0")
p.line(xdat,errfn(xdat),line_color="green") #legend="Fit, dt^4.1")
#p.legend.location = "bottom_right"
#p.legend.click_policy="hide"

show(p) # show the

Loading BokehJS ...

%%html 
<a href="https://github.com/sdorsher/ManyBodyProblem/blob/master/L0%20error%20Circular%20Vector%20RK4%20dt%204p1.png"> L0 error after one circular orbit in the reduced mass sytem </a>

L0 error is the absolute value of the difference between the x coordinate at t=0 and the x coordinate after one orbit. I have plotted the behavior of the L0 error as a function of the timestep dt. For the RK4 numerical differential equation solution method, the error should go as dt^4 or better. It doe, since it goes as dt^4.1. 